In [1]:
import requests

# Wikipedia日本語版のAPIエンドポイント
WIKI_API_URL = "https://ja.wikipedia.org/w/api.php"

# 検索キーワードに該当するWikipediaページを取得する関数
def fetch_search_results(keyword, limit=10):
    params = {
        "action": "query",
        "list": "search",
        "srsearch": keyword,
        "format": "json",
        "srlimit": limit
    }
    
    try:
        response = requests.get(WIKI_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get("query", {}).get("search", [])
    except requests.RequestException as e:
        print("検索リクエストでエラーが発生しました:", e)
        return []

# 指定されたタイトルのページ内容を取得する関数
def fetch_page_content(title):
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }
    
    try:
        response = requests.get(WIKI_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        page = next(iter(data.get("query", {}).get("pages", {}).values()), {})
        return page.get("extract", "ページの内容が見つかりません。")
    except requests.RequestException as e:
        print("ページ内容取得でエラーが発生しました:", e)
        return "ページ内容が取得できませんでした。"

In [2]:
# テストコードで確認
keyword = "Python"

# キーワードに該当するページ一覧を取得
search_results = fetch_search_results(keyword)
if search_results:
    print("【キーワードに該当するページ一覧】")
    for result in search_results[:3]:
        print(f"- {result['title']}")

    # 最初のページの内容を取得
    first_title = search_results[0]['title']
    print(f"\n【最初のページタイトル】")
    print(first_title)

    # 最初のページ内容を取得
    page_content = fetch_page_content(first_title)
    print("\n【ページ内容】")
    print(page_content[:500])
else:
    print("検索結果が見つかりません。")

【キーワードに該当するページ一覧】
- Python
- IronPython
- モンティ・パイソン

【最初のページタイトル】
Python

【ページ内容】
Python（パイソン）はインタープリタ型の高水準汎用プログラミング言語である。


== 概要 ==
Pythonは1991年にグイド・ヴァン・ロッサムにより開発されたプログラミング言語である。
最初にリリースされたPythonの設計哲学は、ホワイトスペース（オフサイドルール）の顕著な使用によってコードの可読性を重視している。その言語構成とオブジェクト指向のアプローチは、プログラマが小規模なプロジェクトから大規模なプロジェクトまで、明確で論理的なコードを書くのを支援することを目的としている。
Pythonは動的に型付けされていて、ガベージコレクションされている。構造化（特に手続き型）、オブジェクト指向、関数型プログラミングを含む複数のプログラミングパラダイムをサポートしている。Pythonは、その包括的な標準ライブラリのため、しばしば「バッテリーを含む」言語と表現されている。
Pythonのインタプリタは多くのOSに対応している。プログラマーのグローバルコミュニティは、自由かつオープンソース  のリファレンス実装であるCPythonを開発および保守している 。非営利団体であるPyth


In [3]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.getenv("API_KEY"))

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [4]:
# 要約を行うプロンプトを作成
prompt = f"""
以下の文章を要約してください。

# 条件：
- 小学生にもわかるように
- 300文字程度

# 文章：
{page_content[:1000]}
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt}
    ],
    max_tokens=500,
    temperature=0.3,   
)

In [5]:
# LLMからの回答を表示
print(response.choices[0].message.content.strip())

Python（パイソン）は、1991年にグイド・ヴァン・ロッサムによって作られたプログラミング言語です。この言語は、コードが読みやすく、書きやすいことを大切にしています。Pythonは、いろいろなプログラミングのスタイルに対応していて、簡単なプログラムから大きなプロジェクトまで使えます。

Pythonの特徴には、動的に型を決めることや、不要になったデータを自動で片付ける「ガベージコレクション」があります。また、Pythonは多くのOSで使えるため、世界中のプログラマーが協力して開発しています。

Pythonの設計思想は、必要な機能だけを提供し、特別な機能はインターネットから追加できるようになっています。これにより、シンプルで効率的なコードを書くことができるのです。Pythonでは「一つの良いやり方がある」という考え方が大切にされています。
